<a href="https://colab.research.google.com/github/songjie-guo/antibody-design/blob/main/Antibody_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare Environments

In [ ]:
import os
import sys
import torch

PYTHON = f"{sys.version_info.major}.{sys.version_info.minor}"

TORCH_version = torch.__version__
TORCH = TORCH_version.split('+')[0]

CUDA_version = torch.version.cuda
CUDA = 'cu' + CUDA_version.replace('.', '')

In [ ]:
os.system("pip install -q 'igfold>=0.3.0'")
os.system("pip install -q --no-warn-conflicts 'py3Dmol>=2.0.1' matplotlib seaborn")

0

In [ ]:
# approximately 42min, can I change it to pip ???
os.system("pip install -q conda")
os.system(f"conda install -y -q -c conda-forge openmm=7.7.0 python='{PYTHON}' pdbfixer 2>&1 1>/dev/null")
os.system(f"conda install -y -q -c bioconda abnumber python='{PYTHON}' 2>&1 1>/dev/null")

256

In [ ]:
os.system(f"pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install -q torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"git clone -q https://github.com/oxpig/SCALOP.git")
os.system(f"pip install SCALOP/")

0

## Prepare files & codes

In [ ]:
import os, sys, shutil
if not os.path.isdir("models"):
    os.system('git clone https://github.com/songjie-guo/antibody-design.git')
    shutil.move('antibody-design/models', 'models')
    shutil.rmtree('antibody-design')

if not os.path.isdir("models/ProDesign"):
    os.chdir('./models')
    os.system("git clone -q https://github.com/A4Bio/ProDesign.git")
    os.makedirs('ProDesign/data/cath', exist_ok=True)
    os.makedirs('ProDesign/data/ts', exist_ok=True)
    os.system('wget -O ProDesign/data/cath.zip https://github.com/A4Bio/PiFold/releases/download/Training%26Data/cath4.2.zip')
    os.system('unzip ProDesign/data/cath.zip -d ProDesign/data/cath')
    os.system('mv ProDesign/data/cath/cath4.2/* ProDesign/data/cath/')
    os.system('wget -O ProDesign/data/ts.zip https://github.com/A4Bio/PiFold/releases/download/Training%26Data/ts.zip')
    os.system('unzip ProDesign/data/ts.zip -d ProDesign/data/')
    os.makedirs('ProDesign/results', exist_ok=True)
    os.system('wget -O ProDesign/results/checkpoint.pth https://github.com/A4Bio/PiFold/releases/download/Training%26Data/checkpoint.pth')
    os.chdir('../')
if not os.path.isdir('cache'):
    os.mkdir('cache')
if not os.path.isdir('results'):
    os.mkdir('results')

In [ ]:
# import importlib

# def is_package_installed(package_name):
#     try:
#         importlib.import_module(package_name)
#         return True
#     except ImportError:
#         return False

# # Example usage:
# if is_package_installed('scalop'):
#     print('scalop is installed')
# else:
#     print('scalop is not installed')

scalop is not installed


In [ ]:
pip list

Package                  Version
------------------------ ----------------
abnumber                 0.3.2
aiohttp                  3.8.4
aiosignal                1.3.1
antiberty                0.1.2
async-timeout            4.0.2
attrs                    23.1.0
biopython                1.81
boltons                  23.0.0
brotlipy                 0.7.0
certifi                  2023.5.7
cffi                     1.15.1
charset-normalizer       2.1.1
cmake                    3.26.4
colorama                 0.4.6
conda                    23.5.0
conda-package-handling   2.0.2
conda_package_streaming  0.7.0
contourpy                1.1.0
cryptography             39.0.2
cycler                   0.11.0
einops                   0.6.1
filelock                 3.12.2
fonttools                4.41.0
frozenlist               1.4.0
fsspec                   2023.6.0
huggingface-hub          0.16.4
idna                     3.4
igfold                   0.3.2
Jinja2                   3.1.2
jsonpatch    

In [ ]:
# if error, please resart the runtime
from igfold.utils.visualize import *
from models.pifold_model import exp
from models.CDR import cut_CDR
from models.predPDB import pred_pdb
from models.predFasta import pred_fasta
from models.replace import replace

ModuleNotFoundError: ignored

In [ ]:
#@title Set Input parameters
H_seq = "QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLEWLGMIWGDGNTDYNSALKSRLSISKDNSKSQVFLKMNSLHTDDTARYYCARERDYRLDYWGQGTTLTVSS" #@param {type:"string"}
L_seq = "DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQLLVYYTTTLADGVPSRFSGSGSGTQYSLKINSLQPEDFGSYYCQHFWSTPRTFGGGTKLEIK" #@param {type:"string"}
num_loops = 5 #@param {type:"integer"}

# cache_path = 'cache'
# result_path = 'results'

num_scheme = 'chothia'
def_scheme = 'chothia'

if_H1 = True #@param {type:"boolean"}
if_H2 = True #@param {type:"boolean"}
if_H3 = True #@param {type:"boolean"}
if_L1 = True #@param {type:"boolean"}
if_L2 = True #@param {type:"boolean"}
if_L3 = True #@param {type:"boolean"}

# igfold model params
# Perform structural refinement with OpenMM
do_refine = True #@param {type:"boolean"}
# Renumber predicted antibody structure (Chothia) with AbNumber
do_renum = False #@param {type:"boolean"}
# Use only a single model for predictions (instead of model ensemble)
single_model = False #@param {type:"boolean"}

## Model

In [ ]:
# create model for pifold
model = exp
# cut H&L
H_cutting,L_cutting = cut_CDR(H_seq,L_seq,num_scheme,def_scheme)

In [ ]:
def run_a_loop(i):
    # for a loop
    sequences, pred = pred_pdb(H_seq,L_seq,do_refine,do_renum,single_model)
    # get_rmsd


    # get_fasta_pred
    H_tmp,L_tmp = pred_fasta(model)
    H_pred,L_pred = replace(H_cutting,L_cutting,H_tmp,L_tmp,
                            if_H1 = True,
                            if_H2 = True,
                            if_H3 = True,
                            if_L1 = True,
                            if_L2 = True,
                            if_L3 = True)
    # store fasta_file
    fasta_file = f'results/pred{i}.fasta'
    with open(fasta_file, "w") as f:
        f.write(f'>:H\n')
        f.write(H_pred+'\n')
        f.write(f'>:L\n')
        f.write(L_pred+'\n')

    # store pdb_file
    if (i-1)>=0:
        import shutil
        src_file = 'cache/tmp.pdb'
        dest_file = f'results/pred{i-1}.pdb'
        shutil.move(src_file, dest_file)

        prmsd_fig_file = f'results/prmsd{i-1}.png'
        plot_prmsd(sequences, pred.prmsd.cpu(), prmsd_fig_file, shade_cdr=do_renum, pdb_file=pred_pdb)

    return H_pred,L_pred

In [ ]:
for i in range(num_loops):
    H_seq,L_seq = run_a_loop(i)